In [25]:
pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import os

In [2]:
os.chdir('C:/Users/mohit/Downloads/Compressed/archive_2')
dir = os.getcwd()
dir

'C:\\Users\\mohit\\Downloads\\Compressed\\archive_2'

In [3]:
train_data = os.path.join('train')
test_data = os.path.join('test')

In [4]:
files = os.listdir(train_data)
print(files[:3])

['angry', 'disgusted', 'fearful']


## Tranfer Learning

In [5]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [6]:
import tensorflow_hub as hub
mobile_net_layers = hub.KerasLayer(mobilenet_v2, input_shape=(224,224,3))

In [7]:
mobile_net_layers.trainable = False

### CNN model for training 

In [5]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

In [6]:
 mod = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax")

In [7]:
ip=mod.layers[0].input
op=mod.layers[-2].output


In [8]:
op

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [9]:
final_output=tf.keras.layers.Dense(1024)(op)
final_output=tf.keras.layers.Activation('relu')(final_output)
final_output=tf.keras.layers.Dropout(0.3)(final_output)
final_output=tf.keras.layers.Dense(64)(final_output)
final_output=tf.keras.layers.Activation('relu')(final_output)
final_output=tf.keras.layers.Dropout(0.3)(final_output)
final_output=tf.keras.layers.Dense(7,activation='softmax')(final_output)

final_output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_2')>

In [10]:
model=tf.keras.Model(inputs =ip,outputs=final_output)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [ ]:
# model = tf.keras.models.Sequential([
    
#     op,
#     tf.keras.layers.Dense(1024 , activation='relu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(512 , activation='relu'),
#     tf.keras.layers.Dense(7 , activation='softmax')
    
# ])

In [ ]:
# model.summary()

In [12]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
             optimizer='Adam',
             metrics=['accuracy'])

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale = 1/255,
#     rotation_range=40,#rotates the image in 0 to 40 degree range randomly
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2, 
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'# lost pixels are replaced by the nearest pixels
)

train_data_gen = train_gen.flow_from_directory(train_data , 
                                              target_size = (224,224),
                                              batch_size=10,
                                              class_mode="categorical")

test_gen = ImageDataGenerator(rescale = 1/255)
test_data_gen = test_gen.flow_from_directory(test_data ,
                                             target_size = (224,224),
                                             batch_size = 10,
                                             class_mode = "categorical")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [14]:
model_fiting = model.fit(train_data_gen , steps_per_epoch=8,epochs=50,validation_data=test_data_gen, verbose=1 ,batch_size =10)

Epoch 1/50
8/8 [==============================] - 100s 12s/step - loss: 2.3569 - accuracy: 0.1875 - val_loss: 2.0104 - val_accuracy: 0.1743
Epoch 2/50
8/8 [==============================] - 79s 11s/step - loss: 1.9449 - accuracy: 0.2000 - val_loss: 2.1717 - val_accuracy: 0.1746
Epoch 3/50
8/8 [==============================] - 79s 11s/step - loss: 1.8222 - accuracy: 0.2750 - val_loss: 2.9394 - val_accuracy: 0.2471
Epoch 4/50
8/8 [==============================] - 79s 11s/step - loss: 1.8722 - accuracy: 0.2750 - val_loss: 3.2433 - val_accuracy: 0.2471
Epoch 5/50
8/8 [==============================] - 79s 11s/step - loss: 2.0119 - accuracy: 0.1750 - val_loss: 2.0044 - val_accuracy: 0.2471
Epoch 6/50
8/8 [==============================] - 79s 11s/step - loss: 1.8764 - accuracy: 0.2250 - val_loss: 1.9217 - val_accuracy: 0.2471
Epoch 7/50
8/8 [==============================] - 79s 11s/step - loss: 1.8363 - accuracy: 0.3250 - val_loss: 1.9359 - val_accuracy: 0.2471
Epoch 8/50
8/8 [==========